<div style="text-align: center; font-weight: 900; font-size: 40px; height: 80px;">Assignment 2 - Electricity Grids</div>

<div style="text-align: right; font-weight: 900;">Yingchen Liu (26981068)</div>

### 1. Using the ADT class for points in 2-dimensional Euclidean space from before

implement a function to generate a test set of n random nodes/points, where n is a user-definable parameter. 

In [185]:
import math

class Node:
    """
    Geo-coded customer (node)
    """
    
    def __init__(self, key, x, y):
        self._key = key
        self._x = x
        self._y = y
        self._parent = self
        self._children = {}
        self._cluster = None
        
    @property
    def key(self):
        return self._key

    @property
    def x(self):
        return self._x
        
    @property
    def y(self):
        return self._y
    
    @property
    def parent(self):
        return self._parent
    
    @parent.setter
    def parent(self, node):
        self._parent = node
        
    @property
    def children(self):
        return self._children
    
    def addChild(self, node):
        self._children[node.key] = node
        
    def removeChild(self, node):
        del self._children[node.key]
        
    @property
    def cluster(self):
        return self.find()._cluster
    
    @cluster.setter
    def cluster(self, cluster):
        self._cluster = cluster
        
    def distanceTo(self, node):
        return math.sqrt(math.pow(node.x - self.x, 2) + math.pow(node.y - self.y, 2))
    
    def union(self, node):
        """
        Union this tree to another node
        
        Parameters:
            node (Node): Node to union to
        """
        self.parent = node
        node.addChild(self)
    
    def find(self):
        """
        Find the root of this tree
        
        Returns:
            Node: The root node
            
        """
        if self == self.parent:
            return self
        else:
            return self.parent.find()
    
    @property
    def descendants(self):
        """
        Get all descendants nodes of this node
        """
        return self._getDescendants([])
    
    def _getDescendants(self, descendants=[]):
        if self._children == {}:
            return descendants
        else:
            for (key, child) in self._children.items():
                descendants.append(child)
                child._getDescendants(descendants)
        return descendants
    
    @property
    def height(self):
        """
        Get the height of this tree from this node
        """
        return self._measureHeight(0)
    
    def _measureHeight(self, height=0):
        if self._children == {}:
            return height
        else:
            childrenHeights = []
            for (key, child) in self._children.items():
                childrenHeights.append(child._measureHeight(height) + 1)
            height += max(childrenHeights)
        return height
    
    def rootify(self, isRoot=True):
        """
        Bring this node to be the root node of this tree
        """
        if not self.parent:
            return
        
        parent = self.parent
        
        if isRoot:
            self.parent = self
        else:
            self.parent = parent
            
        self.addChild(parent)
        parent.removeChild(self)
        
        if parent.parent != self:
            parent.rootify(False)
            
        parent.parent = self
    
    def printPretty(self, indent, isLast):
        """
        Print tree from this node prettily
        """
        print(indent, end='')
        if isLast:
            print('└╴', end='')
            indent += '   '
        else:
            print('├╴', end='')
            indent += '│  '
        print('Node {} ({}, {})'.format(self._key, self._x, self._y))
        
        i = 0
        for key, node in self._children.items():
            node.printPretty(indent, i == len(self._children) - 1)
            i += 1
        
    def __repr__(self):
        return '[Node key={}, parent={}, children={}]'.format(self._key, self._parent.key, self._children)

In [186]:
from random import randint

class Space:
    """
    ADT class for points in 2-dimensional Euclidean space
    """
    
    def __init__(self, minX, maxX, minY, maxY):
        """
        Initialise a space
        
        Parameters:
            minX (int): min x of this area
            maxX (int): max x of this area
            minY (int): min y of this area
            maxY (int): max y of this area
        """
        self._minX = minX
        self._maxX = maxX
        self._minY = minY
        self._maxY = maxY
        self._nodes = {}
        
    def generate(self, n):
        """
        Generate n nodes within in this area
        
        Parameters:
            n (int): Number of nodes
        """
        for i in range(0, n):
            self._nodes[i] = Node(i, randint(self._minX, self._maxX), randint(self._minY, self._maxY))
            
    @property
    def nodes(self):
        """
        Get all nodes in this space
        """
        return self._nodes
    
    def getNode(self, key):
        """
        Get a node with specific key
        
        Parameters:
            key (int): The key
        """
        return self._nodes[key]
            
    def __repr__(self):
        return self._nodes.__repr__()
    

s = Space(0, 100, 0, 100)
s.generate(10)

### 2. Implement an ADT class for Partition (union-find). 

It must support the operations 
* for generating a new partition (i.e. a set of sets), 
* for generating a new set within the partition, 
* for merging two sets (union) and 
* for finding a set to which an element belongs (find). 

You should use the tree-based implementation.

In [187]:
class Cluster:
    """
    Cluster
    """
    
    def __init__(self, key, root):
        """
        Initialise a cluster
        
        Parameters:
            key (int): The key of this cluster
            root (Node): The root node of the tree in this cluster
        """
        self._key = key
        self._root = root
        self._rank = 0
    
    @property
    def key(self):
        return self._key
    
    @key.setter
    def key(self, key):
        self._key = key
        
    @property
    def root(self):
        return self._root
    
    @root.setter
    def root(self, root):
        self._root = root
        
    @property
    def rank(self):
        return self._rank
    
    @rank.setter
    def rank(self, rank):
        self._rank = rank
        
    @property
    def nodes(self):
        """
        Get all nodes in this cluster
        """
        return [self._root] + self._root.descendants
    
    def printPretty(self):
        print('Cluster {}:'.format(self.key))
        self.root.printPretty('', True)
                
    def __repr__(self):
        return '[Cluster key={}, root={}, rank={}]'.format(self._key, self._root, self._rank)

In [188]:
class Edge:
    """
    Edge connects two vertices (customers)
    """
    
    def __init__(self, vertexA, vertexB):
        self._vertexA = vertexA
        self._vertexB = vertexB
        
    @property
    def vertexA(self):
        return self._vertexA
    
    @property
    def vertexB(self):
        return self._vertexB
    
    @property
    def length(self):
        """
        Get the length of this edge
        """
        return self._vertexA.distanceTo(self._vertexB)
    
    def __repr__(self):
        return '[Edge vertexA={}, vertexB={}, length={}]'.format(self.vertexA, self.vertexB, self.length)

In [189]:
class Partition:
    """
    Partition
    """
    
    def __init__(self, nodes):
        """
        Generating a new partition
        
        Parameters:
            nodes ([Node]): Nodes within this partition
        """
        self._clusters = {}
        self._vertices = nodes
        self._edges = {}
        
        vertexList = list(nodes.values())
        for i in range(0, len(vertexList)):
            for j in range(i + 1, len(vertexList)):
                self._edges[(vertexList[i].key, vertexList[j].key)] = Edge(vertexList[i], vertexList[j])
        
    def generateCluster(self, key, root):
        """
        For generating a new set within the partition
        
        Parameters:
            key (int): The key of this cluster
            root (Node): The root node of the tree in this cluster
            
        Returns:
            Cluster: The cluster generated
        """
        root.parent = root
        c = Cluster(key, root)
        self._clusters[key] = c
        root.cluster = c
        return c
        
    def unionCluster(self, nodeA, nodeB):
        """
        For merging two sets (union) from nodes
        
        Parameters:
            nodeA (Node): Node A
            nodeB (Node): Node B
        """
        clusterA = nodeA.cluster
        clusterB = nodeB.cluster
        
        if nodeA.height < nodeB.height:
            nodeA.rootify()
            clusterA.root = nodeA
            
            nodeA.union(nodeB)
            
            if clusterA.key in self._clusters:
                del self._clusters[clusterA.key]
        else:
            nodeB.rootify()
            clusterB.root = nodeB
            
            nodeB.union(nodeA)
            
            if clusterB.key in self._clusters:
                del self._clusters[clusterB.key]
        
    def findCluster(self, node):
        """
        Finding a set to which an element belongs (find)
        
        Parameters:
            node (Node): The element
            
        Returns:
            Cluster: The cluster the element belongs
        """
        return node.find().cluster
    
    def printClusters(self):
        for key, cluster in self._clusters.items():
            cluster.printPretty()
        
    @property
    def clusters(self):
        """
        Get all clusters in the partition
        """
        return self._clusters

         
s = Space(0, 100, 0, 100)
s.generate(10)
p = Partition(s.nodes)

### 3. Implement the clustering procedure described above.

In [190]:
def cluster(self):
    edges = sorted(list(self._edges.values()), key=lambda edge: edge.length)

    for i in range(0, len(self._vertices)):
        self.generateCluster(i, self._vertices[i])

    for i in range(0, len(edges)):
        edge = edges[i]
        clusterA = self.findCluster(edge.vertexA)
        clusterB = self.findCluster(edge.vertexB)
        
        if clusterA != clusterB:
            self.unionCluster(edge.vertexA, edge.vertexB)

    return self._clusters


Partition.cluster = cluster

s = Space(0, 100, 0, 100)
s.generate(10)
p = Partition(s.nodes)

p.cluster()
print()
print('== Result ==========')
p.printClusters()


== Result ==========
Cluster 4:
└╴Node 4 (40, 96)
   ├╴Node 7 (66, 95)
   └╴Node 0 (46, 67)
      ├╴Node 9 (6, 43)
      │  └╴Node 8 (7, 19)
      └╴Node 6 (71, 13)
         └╴Node 5 (69, 7)
            └╴Node 2 (78, 6)
               └╴Node 1 (88, 20)
                  └╴Node 3 (95, 18)


### 4. Extend your forest-based Partition ADT class from above with path compression

In [191]:
def findWithPathCompression(self):
    """
    Find the root of this tree with path compression

    Returns:
        Node: The root node

    """
    if self == self.parent:
        return self
    else:
        root = self.parent.find()
        self.parent.removeChild(self)
        self.parent = root
        root.addChild(self)
        return root

def findClusterWithPathCompression(self, node):
    """
    Finding a set to which an element belongs (find) with path compression

    Parameters:
        node (Node): The element

    Returns:
        Cluster: The cluster the element belongs
    """
    return node.findWithPathCompression().cluster



def clusterWithPathCompression(self):
    edges = sorted(list(self._edges.values()), key=lambda edge: edge.length)

    for i in range(0, len(self._vertices)):
        self.generateCluster(i, self._vertices[i])

    for i in range(0, len(edges)):
        edge = edges[i]
        clusterA = self.findClusterWithPathCompression(edge.vertexA)
        clusterB = self.findClusterWithPathCompression(edge.vertexB)
        
        if clusterA != clusterB:
            self.unionCluster(edge.vertexA, edge.vertexB)
    
    return self._clusters


Node.findWithPathCompression = findWithPathCompression

Partition.findClusterWithPathCompression = findClusterWithPathCompression
Partition.clusterWithPathCompression = clusterWithPathCompression


s = Space(0, 100, 0, 100)
s.generate(10)
p = Partition(s.nodes)

p.clusterWithPathCompression()
print()
print('== Result ==========')
p.printClusters()


== Result ==========
Cluster 1:
└╴Node 1 (91, 77)
   ├╴Node 3 (61, 75)
   ├╴Node 2 (72, 80)
   ├╴Node 0 (76, 82)
   ├╴Node 9 (84, 82)
   ├╴Node 6 (73, 13)
   ├╴Node 5 (1, 73)
   ├╴Node 7 (55, 0)
   ├╴Node 4 (6, 40)
   └╴Node 8 (93, 93)


### 5. Ultimately, we are aiming to find k-clusters 

i.e. we want to the electricity providers to see the structure of the micro-grid if they decide to commission k micro-grids such that all the nodes are covered in the vicinity of these sub-stations. Extend your implementation such that it stops when k clusters have been achieved and return those clusters, where k is a user-definable parameter

In [192]:
def kCluster(self, k):
    edges = sorted(list(self._edges.values()), key=lambda edge: edge.length)

    for i in range(0, len(self._vertices)):
        self.generateCluster(i, self._vertices[i])

    for i in range(0, len(edges)):
        edge = edges[i]
        clusterA = self.findClusterWithPathCompression(edge.vertexA)
        clusterB = self.findClusterWithPathCompression(edge.vertexB)
        
        if clusterA != clusterB:
            self.unionCluster(edge.vertexA, edge.vertexB)

            if len(self._clusters) == min(k, len(self._vertices)):
                break

    return self._clusters


Partition.kCluster = kCluster


s = Space(0, 100, 0, 100)
s.generate(20)
p = Partition(s.nodes)

p.kCluster(3)
print()
print('== Result ==========')
p.printClusters()


== Result ==========
Cluster 3:
└╴Node 3 (57, 24)
   ├╴Node 10 (91, 8)
   ├╴Node 5 (91, 28)
   ├╴Node 18 (79, 29)
   ├╴Node 1 (71, 80)
   │  ├╴Node 4 (81, 91)
   │  ├╴Node 11 (77, 63)
   │  └╴Node 17 (53, 67)
   ├╴Node 16 (48, 85)
   ├╴Node 2 (100, 31)
   ├╴Node 6 (87, 51)
   ├╴Node 7 (76, 23)
   ├╴Node 13 (83, 46)
   └╴Node 0 (41, 42)
      ├╴Node 8 (21, 47)
      ├╴Node 12 (25, 35)
      └╴Node 14 (10, 50)
Cluster 9:
└╴Node 9 (21, 98)
   └╴Node 15 (13, 100)
Cluster 19:
└╴Node 19 (25, 5)


### 6. Implement functions/methods to let the user query whether two given points (nodes) belong to the same cluster (micro-grid).

In [193]:
def isInTheSameClusterWith(self, node):
        """
        Check if this node is in the same cluster with another node
        
        Parameters:
            node (Node): Another node
            
        Returns:
            bool: If they are within the same cluster
        """
        return self.find().cluster == node.find().cluster


def isInTheSameCluster(self, nodeA, nodeB):
    return nodeA.isInTheSameClusterWith(nodeB)


Node.isInTheSameClusterWith = isInTheSameClusterWith

Partition.isInTheSameCluster = isInTheSameCluster


s = Space(0, 100, 0, 100)
s.generate(10)
p = Partition(s.nodes)

p.kCluster(3)

print()
print('== Result ==========')
p.printClusters()

print()
print(0, 1, p.isInTheSameCluster(s.getNode(0), s.getNode(1)))
print(1, 2, p.isInTheSameCluster(s.getNode(1), s.getNode(2)))
print(2, 3, p.isInTheSameCluster(s.getNode(2), s.getNode(3)))
print(3, 4, p.isInTheSameCluster(s.getNode(3), s.getNode(4)))
print(4, 5, p.isInTheSameCluster(s.getNode(4), s.getNode(5)))
print(5, 6, p.isInTheSameCluster(s.getNode(5), s.getNode(6)))
print(6, 7, p.isInTheSameCluster(s.getNode(6), s.getNode(7)))
print(7, 8, p.isInTheSameCluster(s.getNode(7), s.getNode(8)))
print(8, 9, p.isInTheSameCluster(s.getNode(8), s.getNode(9)))
print(9, 0, p.isInTheSameCluster(s.getNode(9), s.getNode(0)))


== Result ==========
Cluster 0:
└╴Node 0 (90, 86)
Cluster 4:
└╴Node 4 (54, 26)
   └╴Node 9 (50, 12)
Cluster 5:
└╴Node 5 (20, 45)
   ├╴Node 8 (7, 54)
   ├╴Node 6 (36, 70)
   │  └╴Node 3 (52, 73)
   │     ├╴Node 7 (53, 97)
   │     └╴Node 1 (73, 56)
   └╴Node 2 (5, 18)

0 1 False
1 2 True
2 3 True
3 4 False
4 5 False
5 6 True
6 7 True
7 8 True
8 9 False
9 0 False


### 7. Define a function to compute the Dunn index, a measure for the quality of the clustering.

In [194]:
class NodeSet:
    """
    Node Set, for calculating closet distance within a set of nodes using divide and conquer approach
    """
    
    def __init__(self, nodes = None):
        if not nodes:
            self._nodes = []
        else:
            self._nodes = nodes
            
    def cons(self, node):
        return NodeSet([node] + self._nodes)
        
    def insert(self, node):
        self._nodes.append(node)
        
    def delete(self, i):
        del self._nodes[i]
        
    def isEmpty(self):
        return self._nodes == []
    
    def first(self):
        if self.isEmpty():
            return None
        else:
            return self._nodes[0]
    
    def last(self):
        if self.isEmpty():
            return None
        else:
            return self._nodes[self.length() - 1]
    
    def rest(self):
        return NodeSet(self._nodes[1:])
    
    def length(self):
        if self.isEmpty():
            return 0
        else: 
            return 1 + self.rest().length()
        
    def __repr__(self):
        return self._nodes.__repr__()
    
    def __iter__(self):
        return self._nodes.__iter__()
    
    def __getitem__(self, key):
        return self._nodes.__getitem__(key)
    
    def mergeX(self, other):
        if self.isEmpty():
            return other
        if other.isEmpty():
            return self

        if self.first().x <= other.first().x:
            return self.rest().mergeX(other).cons(self.first())
        else:
            return self.mergeX(other.rest()).cons(other.first())

    def mergeY(self, other):
        if self.isEmpty():
            return other
        if other.isEmpty():
            return self

        if self.first().y <= other.first().y:
            return self.rest().mergeY(other).cons(self.first())
        else:
            return self.mergeY(other.rest()).cons(other.first())

    def mergeSortX(self):
        if self.length() <= 1:
            return self

        mid = self.length() // 2
        firstHalf = NodeSet(self[:mid])
        lastHalf = NodeSet(self[mid:])
        return firstHalf.mergeSortX().mergeX(lastHalf.mergeSortX())

    def mergeSortY(self):
        if self.length() <= 1:
            return self

        mid = self.length() // 2
        firstHalf = NodeSet(self[:mid])
        lastHalf = NodeSet(self[mid:])
        return firstHalf.mergeSortY().mergeY(lastHalf.mergeSortY())
    
    def minDistanceBruteForce(self):
        dMin = float('Inf')
        for i in range(0, self.length()):
            for j in range(i + 1, self.length()):
                d = self[i].distanceTo(self[j])

                if d < dMin:
                    dMin = d

        return dMin

    def minDistanceDivideAndConquer(self):
        return self.mergeSortX()._minDistanceDivideAndConquer()

    def _minDistanceDivideAndConquer(self):
        if self.length() < 3:
            return self.minDistanceBruteForce()

        mid = self.length() // 2
        leftHalf = NodeSet(self[:mid])
        rightHalf = NodeSet(self[mid:])

        dLeft = leftHalf._minDistanceDivideAndConquer()
        dRight = rightHalf._minDistanceDivideAndConquer()

        if dLeft <= dRight:
            dMin = dLeft
        else:
            dMin = dRight

        midX = leftHalf.last().x
        l = NodeSet(leftHalf._pointsSearchFromRight(midX - dMin)._nodes + 
                     rightHalf._pointsSearchFromLeft(midX + dMin)._nodes).mergeSortY()

        for i in range(0, l.length()):
            dXMin = float('Inf')
            for j in range(i + 1, min(l.length(), i + 16)):
                dX = l[i].distanceTo(l[j])

                if dX < dXMin:
                    dXMin = dX

            if dXMin < dMin:
                dMin = dXMin

        return dMin

    def _pointsSearchFromRight(self, minX):
        if self.isEmpty():
            return NodeSet()

        if self.last().x >= minX:
            return NodeSet(self[:self.length() - 1]) \
                ._pointsSearchFromRight(minX).cons(self.last())
        else:
            return NodeSet(self[:self.length() - 1]) \
                ._pointsSearchFromRight(minX)

    def _pointsSearchFromLeft(self, maxX):
        if self.isEmpty():
            return NodeSet()

        if self.first().x <= maxX:
            return self.rest()._pointsSearchFromLeft(maxX).cons(self.first())
        else:
            return self.rest()._pointsSearchFromLeft(maxX)

In [195]:
def _getMaxDistanceOfCluster(self, cluster):
    edges = []
    for i in range(0, len(cluster.nodes)):
        for j in range(i + 1, len(cluster.nodes)):
            keyA = min(cluster.nodes[i].key, cluster.nodes[j].key)
            keyB = max(cluster.nodes[i].key, cluster.nodes[j].key)
            edges.append(self._edges[(keyA, keyB)])

    if len(edges) > 0:
        edges = sorted(edges, key=lambda edge: -edge.length)
        return edges[0].length
    else:
        return 0

def _getMaxDistance(self):
    distances = []
    for cluster in list(self._clusters.values()):
        distances.append(self._getMaxDistanceOfCluster(cluster))

    return max(distances)

def _getCentroidOfCluster(self, cluster):
    sumX = 0
    sumY = 0
    for node in cluster.nodes:
        sumX += node.x
        sumY += node.y

    return Node(None, sumX / len(cluster.nodes), sumY / len(cluster.nodes))

def _getMinInterClusterDistance(self):
    clusters = list(self._clusters.values())
    if len(clusters) <= 1:
        return 0

    centroids = []
    for cluster in clusters:
        centroids.append(self._getCentroidOfCluster(cluster))

    nodeSet = NodeSet(centroids)
    return nodeSet.minDistanceDivideAndConquer()

def getDunnIndex(self):
    maxDistance = self._getMaxDistance()
    if maxDistance != 0:
        return self._getMinInterClusterDistance() / maxDistance
    else:
        return float('inf')
    

Partition._getMaxDistanceOfCluster = _getMaxDistanceOfCluster
Partition._getMaxDistance = _getMaxDistance
Partition._getCentroidOfCluster = _getCentroidOfCluster
Partition._getMinInterClusterDistance = _getMinInterClusterDistance
Partition.getDunnIndex = getDunnIndex


s = Space(0, 100, 0, 100)
s.generate(10)
p = Partition(s.nodes)

p.kCluster(5)

print()
print('== Result ==========')
p.printClusters()

print()
print('dunnIndex = {}'.format(p.getDunnIndex()))


== Result ==========
Cluster 2:
└╴Node 2 (85, 37)
   ├╴Node 9 (95, 41)
   └╴Node 1 (68, 41)
Cluster 3:
└╴Node 3 (28, 14)
Cluster 4:
└╴Node 4 (60, 94)
Cluster 5:
└╴Node 5 (17, 90)
   └╴Node 6 (14, 69)
      └╴Node 0 (12, 64)
         └╴Node 8 (21, 62)
Cluster 7:
└╴Node 7 (95, 100)

dunnIndex = 1.255487952948972


### 8. Compare the forest that you have generated for the k-clustering to the full minimum cost spanning tree. 

Give a brief, but precise characterization of the sets of edges that are in the MCST but not in the forest of the clustering.

**ANSWER:**

The n edges that are in the MCST but not in the forest of the clustering are the nth longest edges in the MCST.

### Bonus.

You could use Matplotlib or Bokeh to visualize how the algorithm works (i.e. visualize the point sets, the cluster connections as they emerge, and ultimately the results) and even to let the user interactively (graphically) enter a point set (the latter is easier in Matplotlib than in Bokeh).

In [196]:
%matplotlib inline
%config InlineBackend.close_figures = False 

from pylab import *
import matplotlib.pyplot as plt
import numpy as np

from ipywidgets import widgets
from IPython.display import display, clear_output


def kClusterWithVisualisation(self, k, step):
    edges = sorted(list(self._edges.values()), key=lambda edge: edge.length)

    for i in range(0, len(self._vertices)):
        self.generateCluster(i, self._vertices[i])

    for i in range(0, len(edges)):
        edge = edges[i]
        
        if edge.vertexA.cluster != edge.vertexB.cluster:
            self.unionCluster(edge.vertexA, edge.vertexB)

            if i >= step or len(self._clusters) == min(k, len(self._vertices)):
#                 print('Connect node {} and {} with distance {}'.format(edge.vertexA.key, edge.vertexB.key, edge.length))
                return self._clusters, i

            
Partition.kClusterWithVisualisation = kClusterWithVisualisation


def displayConnection(node):
    for child in list(node.children.values()):
        plt.plot([node.x, child.x], [node.y, child.y])
        plt.text((node.x + child.x) / 2, (node.y + child.y) / 2, float('{:.2f}'.format(node.distanceTo(child))), fontdict=fontDistance)
        displayConnection(child)


rcParams['figure.figsize'] = (20, 20)

fontDistance = {
    'family': 'serif',
    'color': 'darkred',
    'weight': 'normal',
    'size': 8
}
fontNode = {
    'family': 'serif',
    'color': 'darkred',
    'weight': 'normal',
    'size': 10
}
fontCluster = {
    'family': 'serif',
    'color': 'darkred',
    'weight': 'normal',
    'size': 12
}

plt.ioff()
fig, ax = plt.subplots()


sliderNodes = widgets.IntSlider(
    value=40,
    min=1,
    max=100,
    step=1,
    description='Nodes:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(sliderNodes)

sliderClusters = widgets.IntSlider(
    value=3,
    min=1,
    max=10,
    step=1,
    description='Clusters:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(sliderClusters)

out = widgets.Output()
startButton = widgets.Button(description='Start Simulation')
vbox = widgets.VBox(children=(out, startButton))
display(vbox)

step = -1
s = Space(0, 100, 0, 100)

def nextButtonClicked(b):
    global step
    plt.cla()

    for point in list(s.nodes.values()):
        plt.plot(point.x, point.y, marker='o', markersize=3, color="red")
        plt.text(point.x, point.y, point.key, fontdict=fontNode)

    clusters = {}
    if step >= 0:
        p = Partition(s.nodes)
        clusters, step = p.kClusterWithVisualisation(sliderClusters.value, step)

        for cluster in list(clusters.values()):
            plt.text(cluster.root.x - 2, cluster.root.y + 3, 'Cluster {}'.format(cluster.key), fontdict=fontCluster)
            plt.text(cluster.root.x - 2, cluster.root.y + 2, 'height={}'.format(cluster.root.height), fontdict=fontNode)
            displayConnection(cluster.root)

    with out:
        clear_output(wait=True)
        display(ax.figure)

    if len(clusters) == sliderClusters.value:
        print()
        print('== Result ==========')
        p.printClusters()

        print()
        print('dunnIndex = {}'.format(p.getDunnIndex()))
        return

    step = step + 1

def startButtonClicked(b):
    global step
    
    startButton.close()
    sliderNodes.close()
    sliderClusters.close()
    
    # Generate nodes
    
    s.generate(sliderNodes.value)
    
    step = -1

    out = widgets.Output()
    button = widgets.Button(description='Next')
    vbox = widgets.VBox(children=(out, button))
    display(vbox)

    button.on_click(nextButtonClicked)
    nextButtonClicked(None)

startButton.on_click(startButtonClicked)